In [2]:
import torch
from glob import glob

# Loading trained model 

yolov7_path = '../'
model_path = '/home/jesus/Environment/malaria/yolov7/runs/train/x-500-batch-4/weights/best.pt'

model = torch.hub.load(yolov7_path,
                       'custom', 
                       path_or_model=model_path, 
                       force_reload=True, 
                       source='local')


                 from  n    params  module                                  arguments                     
  0                -1  1      1160  models.common.Conv                      [3, 40, 3, 1]                 
  1                -1  1     28960  models.common.Conv                      [40, 80, 3, 2]                
  2                -1  1     57760  models.common.Conv                      [80, 80, 3, 1]                
  3                -1  1    115520  models.common.Conv                      [80, 160, 3, 2]               
  4                -1  1     10368  models.common.Conv                      [160, 64, 1, 1]               
  5                -2  1     10368  models.common.Conv                      [160, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

 76                28  1    102720  models.common.Conv                      [640, 160, 1, 1]              
 77          [-1, -2]  1         0  models.common.Concat                    [1]                           
 78                -1  1     41216  models.common.Conv                      [320, 128, 1, 1]              
 79                -2  1     41216  models.common.Conv                      [320, 128, 1, 1]              
 80                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 81                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 82                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 83                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 84                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 85                -1  1    147712  m

Adding autoShape... 


In [88]:
# Get result of a detection using an image path
def detectImage (img_path):
    inference = model(img_path)
    
    return inference

# Check if the box is small or not
def isSmallBox(side1, side2):
    if(side1 <= 32 and side2 <= 32):
        return True
    elif (side1 * side2 <= 32 * 32):
        return True
    else:
        return False

# Count all boxes and small boxes of a detection
def countBoxesInDetection (results):
    predicts = results.to_dict(orient='records')
    
    small_counter = 0
    all_counter = len(predicts)
    
    for p in predicts:
        side1 = p['xmax'] - p['xmin']
        side2 = p['ymax'] - p['ymin']

        if(isSmallBox(side1, side2)) :
            small_counter = small_counter + 1
    
    return [all_counter, small_counter]


# Return the result of boxes in a image
def getBoxesInImage(img_path):
    detection = detectImage(img_path)
    
    detection = detection.pandas().xyxy[0]
    
    boxes = countBoxesInDetection(detection)

    return boxes

# Return all boxes and small boxes of a partition from the dataset
def getBoxesInPartition(partition='train'):
    directory = '/home/jesus/Environment/malaria/thick-blood-smears-150-YOLO/images/' + partition

    images = glob(directory + "/*.jpg", recursive = True)
    
    all_boxes = 0
    small_boxes = 0
    
    print('running...');
    
    for img in images:
        boxes = getBoxesInImage(img)
        
        all_boxes = all_boxes + boxes[0]
        small_boxes = small_boxes + boxes[1]

    return [all_boxes, small_boxes]
    

In [89]:
results = getBoxesInPartition('train')

print(results)

running...
[57538, 20]


In [90]:
results = getBoxesInPartition('val')

print(results)

running...
[6524, 0]


In [91]:
results = getBoxesInPartition('test')

print(results)

running...
[20398, 0]
